# Introduction: Data Merging

This Jupyter Notebook outlines a data processing pipeline for merging multiple datasets related to Wikipedia articles, US cities, and US states by region. The ultimate goal is to create a comprehensive dataset containing information about the regional divisions, populations, Wikipedia article details, and ORES predictions for various US states.


In [163]:
# Import necessary libraries
import pandas as pd

# Step 1: Reading the datasets
In this step, we read three separate datasets: 'cleaned_data.csv', 'us_cities_by_state_SEPT.2023.csv', and 'US States by Region - US Census Bureau.xlsx'. These datasets contain crucial information about ORES predictions, US city details, and US state divisions, respectively.


In [164]:
## Reading the 'ores_predictions.csv' file
ores_df = pd.read_csv('../data/cleaned_data.csv') 

## Reading the 'us_cities_by_state_SEPT.2023.csv' file
cities_df = pd.read_csv('../data/us_cities_by_state_SEPT.2023.csv')

## Reading the 'US States by Region - US Census Bureau.xlsx' file
regions_df = pd.read_excel('../data/US States by Region - US Census Bureau.xlsx')

# Read the NST-EST2022-POP.xlsx file
population_df = pd.read_excel('../data/NST-EST2022-POP.xlsx', header=[2,3])

ores_df

,title,rev_id,prediction
0,"Abbeville, Alabama",1171163550,C
1,"Adamsville, Alabama",1177621427,C
2,"Addison, Alabama",1168359898,C
3,"Akron, Alabama",1165909508,GA
4,"Alabaster, Alabama",1179139816,C
...,...,...,...
22118,"Wright, Wyoming",1166334449,GA
22119,"Yoder, Wyoming",1171182284,C
22120,"Jennings, Missouri",1165751175,C
22121,"Jefferson Township, Greene County, Pennsylvania",1171582274,C


Removing dublicates from the given df

In [165]:
# Assuming 'common_key_column' is the column based on which you want to identify unique rows
ores_df.drop_duplicates(subset='title', keep='first', inplace=True)
cities_df.drop_duplicates(subset='page_title', keep='first', inplace=True)

ores_df

,title,rev_id,prediction
0,"Abbeville, Alabama",1171163550,C
1,"Adamsville, Alabama",1177621427,C
2,"Addison, Alabama",1168359898,C
3,"Akron, Alabama",1165909508,GA
4,"Alabaster, Alabama",1179139816,C
...,...,...,...
22118,"Wright, Wyoming",1166334449,GA
22119,"Yoder, Wyoming",1171182284,C
22120,"Jennings, Missouri",1165751175,C
22121,"Jefferson Township, Greene County, Pennsylvania",1171582274,C


# Step 2: Data Preprocessing
This step involves necessary data preprocessing tasks. We extract the 'State' column from the 'us_cities_by_state_SEPT.2023.csv' dataset. Additionally, we merge the 'ores_df' and 'cities_df' dataframes to combine relevant information from both datasets.


In [166]:
## Extracting the 'State' column from the cities dataframe
cities_df = cities_df[['page_title', 'state']]

## Merging the 'ores_df' and 'cities_df' on the 'Title' and 'page_title' columns
merged_df = pd.merge(ores_df, cities_df, left_on='title', right_on='page_title', how='left')



In [167]:
merged_df

,title,rev_id,prediction,page_title,state
0,"Abbeville, Alabama",1171163550,C,"Abbeville, Alabama",Alabama
1,"Adamsville, Alabama",1177621427,C,"Adamsville, Alabama",Alabama
2,"Addison, Alabama",1168359898,C,"Addison, Alabama",Alabama
3,"Akron, Alabama",1165909508,GA,"Akron, Alabama",Alabama
4,"Alabaster, Alabama",1179139816,C,"Alabaster, Alabama",Alabama
...,...,...,...,...,...
21503,"Wright, Wyoming",1166334449,GA,"Wright, Wyoming",Wyoming
21504,"Yoder, Wyoming",1171182284,C,"Yoder, Wyoming",Wyoming
21505,"Jennings, Missouri",1165751175,C,"Jennings, Missouri",Missouri
21506,"Jefferson Township, Greene County, Pennsylvania",1171582274,C,"Jefferson Township, Greene County, Pennsylvania",Pennsylvania


# Step 3: Merging the Dataframes
Here, we merge the previously combined dataframe with the 'regions_df' dataframe based on the 'State' column. This results in a comprehensive dataframe containing data from all three initial datasets, providing a holistic view of the US states, their regions, and corresponding Wikipedia article details.


In [168]:
## Merging the 'merged_df' and 'regions_df' on the 'State' column
# Preprocessing the 'final_df' dataframe for standardizing state values
final_df['state'] = merged_df['state'].str.strip().str.lower()

# Filling the NaN values in the 'REGION' and 'DIVISION' columns with the last valid observation
regions_df['REGION'] = regions_df['REGION'].fillna(method='ffill')
regions_df['DIVISION'] = regions_df['DIVISION'].fillna(method='ffill')

# Preprocessing the 'regions_df' dataframe for standardizing state values and formatting the divisions
regions_df['STATE'] = regions_df['STATE'].str.strip().str.lower()
regions_df['DIVISION'] = regions_df['DIVISION'].apply(lambda x: x.strip() if isinstance(x, str) else x)


regions_df

/var/folders/27/xxn299vj48dfm1rmr1s911c00000gn/T/ipykernel_8075/1739781989.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['state'] = merged_df['state'].str.strip().str.lower()


,REGION,DIVISION,STATE
0,Northeast,NaN,NaN
1,Northeast,New England,NaN
2,Northeast,New England,connecticut
3,Northeast,New England,maine
4,Northeast,New England,massachusetts
...,...,...,...
58,West,Pacific,alaska
59,West,Pacific,california
60,West,Pacific,hawaii
61,West,Pacific,oregon


In [169]:

merged_df['state'] = merged_df['state'].str.lower()
regions_df['STATE'] = regions_df['STATE'].str.lower()

# Merging the 'final_df' and 'regions_df' on the 'state' column
final_df = pd.merge(merged_df, regions_df, left_on='state', right_on='STATE', how='left')

final_df

,title,rev_id,prediction,page_title,state,REGION,DIVISION,STATE
0,"Abbeville, Alabama",1171163550,C,"Abbeville, Alabama",alabama,South,East South Central,alabama
1,"Adamsville, Alabama",1177621427,C,"Adamsville, Alabama",alabama,South,East South Central,alabama
2,"Addison, Alabama",1168359898,C,"Addison, Alabama",alabama,South,East South Central,alabama
3,"Akron, Alabama",1165909508,GA,"Akron, Alabama",alabama,South,East South Central,alabama
4,"Alabaster, Alabama",1179139816,C,"Alabaster, Alabama",alabama,South,East South Central,alabama
...,...,...,...,...,...,...,...,...
21503,"Wright, Wyoming",1166334449,GA,"Wright, Wyoming",wyoming,West,Mountain,wyoming
21504,"Yoder, Wyoming",1171182284,C,"Yoder, Wyoming",wyoming,West,Mountain,wyoming
21505,"Jennings, Missouri",1165751175,C,"Jennings, Missouri",missouri,Midwest,West North Central,missouri
21506,"Jefferson Township, Greene County, Pennsylvania",1171582274,C,"Jefferson Township, Greene County, Pennsylvania",pennsylvania,Northeast,Middle Atlantic,pennsylvania


# Step 4: Selecting the Required Columns
To streamline the dataset, we select only the necessary columns, including 'State', 'DIVISION', 'article_title', 'Last_Revision_ID', and 'Prediction'. This ensures that the resulting dataset remains focused on the essential information for further analysis.


In [170]:
# Selecting the necessary columns for the final dataset
final_df = final_df[['state', 'DIVISION',  'title', 'rev_id', 'prediction']]



# Step 5: Renaming the Columns
To improve the readability of the final dataset, we rename the columns to more intuitive and descriptive names, providing a clearer understanding of the data contained within the dataset.


In [171]:
# Renaming the columns for better readability
final_df.columns = ['state', 'regional_division', 'article_title', 'revision_id', 'article_quality']
final_df

,state,regional_division,article_title,revision_id,article_quality
0,alabama,East South Central,"Abbeville, Alabama",1171163550,C
1,alabama,East South Central,"Adamsville, Alabama",1177621427,C
2,alabama,East South Central,"Addison, Alabama",1168359898,C
3,alabama,East South Central,"Akron, Alabama",1165909508,GA
4,alabama,East South Central,"Alabaster, Alabama",1179139816,C
...,...,...,...,...,...
21503,wyoming,Mountain,"Wright, Wyoming",1166334449,GA
21504,wyoming,Mountain,"Yoder, Wyoming",1171182284,C
21505,missouri,West North Central,"Jennings, Missouri",1165751175,C
21506,pennsylvania,Middle Atlantic,"Jefferson Township, Greene County, Pennsylvania",1171582274,C


# Step 6: Saving the Final Dataset
Finally, we save the resulting dataset to a CSV file named 'resulting_data.csv'. This file contains all the essential information merged from the initial datasets, offering valuable insights into US state regional divisions, populations, Wikipedia article details, and ORES predictions.


In [172]:

# Saving the resulting data to a CSV file
final_df.to_csv('../data/resulting_data.csv', index=False)


# Display the first few rows of the final merged dataset
final_df.head()


,state,regional_division,article_title,revision_id,article_quality
0,alabama,East South Central,"Abbeville, Alabama",1171163550,C
1,alabama,East South Central,"Adamsville, Alabama",1177621427,C
2,alabama,East South Central,"Addison, Alabama",1168359898,C
3,alabama,East South Central,"Akron, Alabama",1165909508,GA
4,alabama,East South Central,"Alabaster, Alabama",1179139816,C


# Step 7: Merging the population dataset
Now we have the regional divion information from the dataset, we need to collect the population information as well. For this we will be merging NST-EST2022-POP.xlsx. For more information, please read readme.

For better clearity purpose, the data is manually cleaned.

In [173]:
# Remove any leading dots in the 'Geographic Area' column to match the 'state' column
population_df.columns = ['Geographic Area', 'April 1, 2020', '2020', '2021', '2022']


In [174]:
population_df = population_df[['Geographic Area', '2022']]
population_df = population_df.rename(columns={'2022': 'Population'})

population_df

,Geographic Area,Population
0,Northeast,57040406.0
1,Midwest,68787595.0
2,South,128716192.0
3,West,78743364.0
4,.Alabama,5074296.0
5,.Alaska,733583.0
6,.Arizona,7359197.0
7,.Arkansas,3045637.0
8,.California,39029342.0
9,.Colorado,5839926.0


In [175]:
population_df.loc[:, 'Geographic Area'] = population_df['Geographic Area'].str.replace(r'^\W+', '', regex=True).str.lower()
population_df

,Geographic Area,Population
0,northeast,57040406.0
1,midwest,68787595.0
2,south,128716192.0
3,west,78743364.0
4,alabama,5074296.0
5,alaska,733583.0
6,arizona,7359197.0
7,arkansas,3045637.0
8,california,39029342.0
9,colorado,5839926.0


In [176]:
merged_data_df = pd.merge(final_df, population_df, left_on='state', right_on='Geographic Area', how='left')

merged_data_df



,state,regional_division,article_title,revision_id,article_quality,Geographic Area,Population
0,alabama,East South Central,"Abbeville, Alabama",1171163550,C,alabama,5074296.0
1,alabama,East South Central,"Adamsville, Alabama",1177621427,C,alabama,5074296.0
2,alabama,East South Central,"Addison, Alabama",1168359898,C,alabama,5074296.0
3,alabama,East South Central,"Akron, Alabama",1165909508,GA,alabama,5074296.0
4,alabama,East South Central,"Alabaster, Alabama",1179139816,C,alabama,5074296.0
...,...,...,...,...,...,...,...
21503,wyoming,Mountain,"Wright, Wyoming",1166334449,GA,wyoming,581381.0
21504,wyoming,Mountain,"Yoder, Wyoming",1171182284,C,wyoming,581381.0
21505,missouri,West North Central,"Jennings, Missouri",1165751175,C,missouri,6177957.0
21506,pennsylvania,Middle Atlantic,"Jefferson Township, Greene County, Pennsylvania",1171582274,C,pennsylvania,12972008.0


In [181]:
# Removing the 'Geographic_Area' column
merged_data_df.drop('Geographic Area', axis=1, inplace=True)

# Changing the name of the 'Population' column to 'population'
merged_data_df.rename(columns={'Population': 'population'}, inplace=True)


In [182]:
# Save the merged DataFrame to a new CSV file
merged_data_df.to_csv('../data/wp_scored_city_articles_by_state.csv', index=False)